# Latest Product Table

- p_id from **1 to 100** already present in  main table, it can be updated and deleted
- p_id from **101 to 200** can be inserted in to main table and after inserting it into main table it could also be deleted


### Libraries Used

In [1]:
import datetime
import os

# Main entry point for DataFrame and SQL functionality.
from pyspark.sql import SparkSession
# Start SPARK Session
spark = SparkSession.builder.appName('Basics').getOrCreate()

from pyspark.sql.functions import *
from pyspark.sql.types import *

### Read the Main Table

In [2]:
# Read from main_table
Table = spark.read.format('csv').options(
    header=True, inferschema=True).load(
        "/home/bluepi/Downloads/Update/product_info/main table/main_table_new.csv")

# Add record type to main table
Table_new = Table.withColumn('record_type', lit("A"))

# Write in main_table folder
path = "/home/bluepi/Downloads/Update/Updated Product/Latest Product/"
try:
    Table_new.write.format('csv').save(
        os.path.join(path, 'main_table'), header=True)
except:
    Table_new.write.mode('overwrite').format('csv').save(
        os.path.join(path, 'main_table'), header=True)

In [3]:
# GFG
class my_dictionary(dict):

    # __init__ function
    def __init__(self):
        self = dict()

    # Function to add key:value
    def add(self, key, value):
        self[key] = value

### Main CODE

In [4]:
# Storing all the "p_id as value" in a dictionary with "date as key", which we have inserted,deleted & updated            
total_products_updated = my_dictionary()
total_products_inserted = my_dictionary()
total_products_deleted = my_dictionary()

# Storing all the p_id in a list which we have deleted and inserted
total_products_inserted_list = []
total_products_deleted_list = []

# ###################################################################################################

for i in range(10, 0, -1):

    data_schema = [
        StructField('p_id', IntegerType(), True),
        StructField('p_name', StringType(), True),
        StructField('price', IntegerType(), True),
        StructField('date_timestamp', TimestampType(), True),
        StructField('record_type', StringType(), True)
    ]
    # Schema which we are accepting
    final_struc = StructType(fields=data_schema)

    # In this try-catch block we are only taking the previous day data, but initially there is
    # no previous day, therefore we are taking main table at begining
    try:
        path1 = os.path.join(
            "/home/bluepi/Downloads/Update/Updated Product/Latest Product/", t)
        try:
            os.remove(os.path.join(path1, '_SUCCESS'))
        except:
            pass

        mainTable_old = spark.read.csv(path1, header=True, schema=final_struc)
        mainTable = spark.createDataFrame([], schema=final_struc)
        mainTable = mainTable.union(mainTable_old)
    except:
        path1 = os.path.join(
            "/home/bluepi/Downloads/Update/Updated Product/Latest Product/", "main_table")
        try:
            os.remove(os.path.join(path1, '_SUCCESS'))
        except:
            pass
        mainTable_old = spark.read.csv(path1, header=True, schema=final_struc)
        mainTable = spark.createDataFrame([], schema=final_struc)
        mainTable = mainTable.union(mainTable_old)

    print("\n File read from --")
    print(path1, end='\n')

    # Address to the product_info folder
    address = "/home/bluepi/Downloads/Update/product_info/"
    previous_day = (datetime.datetime.today() -
                    datetime.timedelta(days=i)).strftime('%d-%m-%Y')
    print("\n\t\ Date ----> "+previous_day+"\n")

    # before_insert = str(mainTable.count())
    print("Main_Table Count ----> " + str(mainTable.count()))

    # Address to the Previous Day folder
    new_address = address + previous_day
#     print("\nNew Address to read the folder ---->"+new_address)

# ###################################################################################################

    # Read the Previous Day folder
    per_day_data = spark.read.format('csv') \
        .options(header=True, inferschema=True) \
        .load(new_address)

    # Get the list of p_id which we have to inserted in the main table
    to_be_inserted = per_day_data.filter("record_type == 'I' ").collect()
    p_id_list_I = [i.p_id for i in to_be_inserted]
    
    # Since we don't want our Inserted product to be Inserted again
    for i in total_products_inserted_list :
        if i in p_id_list_I :
            print("\n{} is going to be inserted again\n".format(i))
            # Therefore removing already inserted products
            p_id_list_I.remove(i)
    # Storing all the p_id in a list which we have inserted
    total_products_inserted_list.extend(p_id_list_I)

    # Directly append new Inserted products
    mainTable_I_inserted = mainTable.union(
        per_day_data.filter("record_type == 'I' "))

    after_insert = str(mainTable_I_inserted.count())
    total_insert = str(per_day_data.filter("record_type == 'I' ").count())

    print("Total \"Inserted I\"----> " + total_insert)
    print("After Inserting----> " + after_insert)

    # Storing all the "p_id as value" in a dictionary with "date as key", which we have inserted
    total_products_inserted.add(previous_day, p_id_list_I)
    p_id_list_I.sort()
    print(" ".join(map(str, p_id_list_I)))
    print("\n")

# ###################################################################################################

# Created a new DataFrame of records to be updated
    from_per_day_data_U = per_day_data.filter("record_type == 'U' ")
    
# Get the list of p_id which we have to update taken from dated folders
    from_per_day_data_U_list = from_per_day_data_U.select("p_id").collect()

# List comprehension
    p_id_list_U = [i.p_id for i in from_per_day_data_U_list]
    p_id_list_U.sort()
    print("TO Be Update\"per_day_data\"")
    print(" ".join(map(str, p_id_list_U)))
#     print(p_id_list_U)

    # Storing all the "p_id as value" in a dictionary with "date as key", which we have updated
    total_products_updated.add(previous_day, p_id_list_U)

    total_update = str(len((p_id_list_U)))
    print("Update Count ------>"+total_update)
    print("\n")
    
# Get the products which we have to update, present in Main Table
    from_mainTable_U = mainTable_I_inserted.filter(
        col('p_id').isin(p_id_list_U))

    # Performed Union operation on the above tables
    mT_and_pDD_union = from_mainTable_U.union(from_per_day_data_U)
#     mT_and_pDD_union.orderBy(mT_and_pDD_union.p_id).show()

    # Performed GroupBy operation on P_ID and take the latest date only
    #    Rename the columns
    for_join_mT_and_pDD = mT_and_pDD_union.groupBy("p_id").agg(
        {"date_timestamp": "max"}).withColumnRenamed("max(date_timestamp)", "date_timestamp_1")
    for_join_mT_and_pDD = for_join_mT_and_pDD.withColumnRenamed(
        "p_id", "p_id_1")
#     for_join_mT_and_pDD.show()

    # Performed Join operation to pick only latest updates only
    joined = mT_and_pDD_union.join(for_join_mT_and_pDD, (
        mT_and_pDD_union.p_id == for_join_mT_and_pDD.p_id_1) & (
            mT_and_pDD_union.date_timestamp == for_join_mT_and_pDD.date_timestamp_1), 'inner')

    joined = joined.select(
        ['p_id', 'p_name', 'price', 'date_timestamp', 'record_type'])
#     joined.count()

    # First remove the p_id from Main_Table which we have to updated
    mainTable_U_updated = mainTable_I_inserted.filter(
        ~col('p_id').isin(p_id_list_U))
#     mainTable_U_updated.orderBy("p_id").count()

    # Then Add the Updated P_ID to the Main_Table
    mainTable_U_updated_new = mainTable_U_updated.union(joined)
#     after_update = str(mainTable_U_updated_new.orderBy("p_id").count())
#     mainTable_U_updated_new.orderBy("p_id").count()

# ###################################################################################################

    # Get the list of p_id which we have to delete
    to_be_deleted = per_day_data.filter("record_type == 'D' ").collect()
    p_id_list_D = [i.p_id for i in to_be_deleted]
    
    # Since we don't want our deleted product to be deleted again
    for i in total_products_deleted_list :
        if i in p_id_list_D :
            print("\n{} is going to be deleted again\n".format(i))
            # Therefore removing already deleted products
            p_id_list_D.remove(i)
    # Storing all the p_id in a list which we have deleted            
    total_products_deleted_list.extend(p_id_list_D)
            
    
    # Storing all the "p_id as value" in a dictionary with "date as key", which we have deleted
    total_products_deleted.add(previous_day, p_id_list_D)
    

#     print("\nList of p_id which we have to deleted taken from \"per_day_data\"")
#     print(p_id_list_D)
    total_deleted = str(len((p_id_list_D)))
    p_id_list_D.sort()
    print("\nTotal Deleted ------>"+str(len((p_id_list_D))))
    print(" ".join(map(str, p_id_list_D)))

    # Remove the deleted p_id from main_table
    mainTable_D_deleted = mainTable_U_updated_new.filter(
        ~col('p_id').isin(p_id_list_D))

    after_delete = str(mainTable_D_deleted.count())
    print("After Deleting Count---->"+after_delete)
    

# ###################################################################################################

    # This writes the DF in different files becaues of parallism
    t = "main_table"+str(previous_day)
#     print(t,end="\n")


    mainTable_D_deleted.write.format('csv').save(
            os.path.join(path, t), header=True)

# Since, coalesce opertation might become costly when working with large files, therefore not using it.
#     mainTable_D_deleted.coalesce(1).write.format('csv').save(
#             os.path.join(path, t), header=True)


    print("\n Main Table Stored ")
    print(os.path.join(path, t))

    print("\n***************************************************************************************\n")
#     input("Press Enter to continue...")


 File read from --
/home/bluepi/Downloads/Update/Updated Product/Latest Product/main_table

	\ Date ----> 05-04-2020

Main_Table Count ----> 100
Total "Inserted I"----> 9
After Inserting----> 109
108 112 116 118 134 147 155 173 185


TO Be Update"per_day_data"
2 5 6 8 14 15 25 36 39 41 44 57 59 60 63 63 68 71 71 81 82 83 84 85 86 92 95 95 97 134 185
Update Count ------>31



Total Deleted ------>10
9 20 30 40 47 61 63 70 72 100
After Deleting Count---->99

 Main Table Stored 
/home/bluepi/Downloads/Update/Updated Product/Latest Product/main_table05-04-2020

***************************************************************************************


 File read from --
/home/bluepi/Downloads/Update/Updated Product/Latest Product/main_table05-04-2020

	\ Date ----> 06-04-2020

Main_Table Count ----> 99
Total "Inserted I"----> 10
After Inserting----> 109
105 109 123 139 166 167 175 176 178 186


TO Be Update"per_day_data"
7 15 15 31 51 55 56 57 64 75 78 83 92 96
Update Count ------>14



Tot

### Products Updated in a day

In [5]:
l = mT_and_pDD_union.groupBy("p_id").agg({'p_id':'count'}).filter("count(p_id)>1").select('p_id').collect()
l1 = [ i.p_id for i in l]
mT_and_pDD_union.filter( col('p_id').isin(l1) ).orderBy(col('p_id')).show()

+----+---------+-----+-------------------+-----------+
|p_id|   p_name|price|     date_timestamp|record_type|
+----+---------+-----+-------------------+-----------+
|   8| Alphazap| 1044|2020-04-11 15:29:28|          U|
|   8| Alphazap|  672|2020-04-13 03:16:18|          U|
|  39|   Sonair|  186|2020-04-13 03:27:38|          U|
|  39|   Sonair| 1078|2020-04-12 21:25:17|          U|
|  42|  Andalax| 1276|2020-04-07 09:26:57|          U|
|  42|  Andalax|  889|2020-04-13 03:43:08|          U|
|  66|Gembucket|  611|2020-04-13 21:20:37|          U|
|  66|Gembucket|  764|2020-04-12 21:45:42|          U|
|  71|   Keylex|  523|2020-04-08 21:44:27|          U|
|  71|   Keylex| 1131|2020-04-13 03:15:40|          U|
|  88|  Redhold|  748|2020-04-09 15:42:24|          U|
|  88|  Redhold|  721|2020-04-13 03:39:50|          U|
| 152|  Prodder|  110|2020-04-10 15:14:29|          U|
| 152|  Prodder|  299|2020-04-13 09:45:37|          U|
| 155| Bytecard|  239|2020-04-13 21:13:14|          U|
| 155| Byt

### Final Main Table

In [6]:
mainTable_D_deleted.filter( col('p_id').isin(l1) ).orderBy(col('p_id')).show()

+----+---------+-----+-------------------+-----------+
|p_id|   p_name|price|     date_timestamp|record_type|
+----+---------+-----+-------------------+-----------+
|   8| Alphazap|  672|2020-04-13 03:16:18|          U|
|  39|   Sonair|  186|2020-04-13 03:27:38|          U|
|  42|  Andalax|  889|2020-04-13 03:43:08|          U|
|  66|Gembucket|  611|2020-04-13 21:20:37|          U|
|  71|   Keylex| 1131|2020-04-13 03:15:40|          U|
|  88|  Redhold|  721|2020-04-13 03:39:50|          U|
| 152|  Prodder|  299|2020-04-13 09:45:37|          U|
| 155| Bytecard|  239|2020-04-13 21:13:14|          U|
| 157|   Lotlux| 1191|2020-04-13 15:43:26|          U|
+----+---------+-----+-------------------+-----------+



### Total Insertion, Updation & Deletion Day Wise

In [7]:
import pprint
# prints the formatted representation of PrettyPrinter object
pp = pprint.PrettyPrinter(indent=4)

for i in range(10,0,-1) :
    previous_day = (datetime.datetime.today() - datetime.timedelta(days=i)).strftime('%d-%m-%Y')
    pp.pprint(previous_day)
    print("inserted :\t")
    print(' '.join(map(str, total_products_inserted.get(previous_day))))
#     pp.pprint(total_products_inserted.get(previous_day))
    print("updated :\t")
    print(' '.join(map(str, total_products_updated.get(previous_day))))
#     pp.pprint(total_products_updated.get(previous_day))
    print("deleted :\t")
    print(' '.join(map(str, total_products_deleted.get(previous_day))))
#     pp.pprint(total_products_deleted.get(previous_day))
    print("\n")

'05-04-2020'
inserted :	
108 112 116 118 134 147 155 173 185
updated :	
2 5 6 8 14 15 25 36 39 41 44 57 59 60 63 63 68 71 71 81 82 83 84 85 86 92 95 95 97 134 185
deleted :	
9 20 30 40 47 61 63 70 72 100


'06-04-2020'
inserted :	
105 109 123 139 166 167 175 176 178 186
updated :	
7 15 15 31 51 55 56 57 64 75 78 83 92 96
deleted :	
3 11 94 98 116


'07-04-2020'
inserted :	
101 102 110 117 128 144 157 162 179 180 182 188 192
updated :	
2 3 8 16 28 32 41 49 53 56 71 76 86 90 92 105 108 162 166 167
deleted :	
13 18 23 56 62 68 180


'08-04-2020'
inserted :	
104 111 120 121 141 148 152 170 171
updated :	
6 7 15 16 17 29 35 42 46 48 53 55 71 74 85 95 141 157 176 179 182
deleted :	
69


'09-04-2020'
inserted :	
122 129 130 133 150 156 159 174 174 189 197
updated :	
3 7 37 53 54 54 57 60 68 71 75 76 77 82 82 85 95 101 175 182 185 185
deleted :	
2 78 81 86 173


'10-04-2020'
inserted :	
106 107 113 114 125 140 142 149 177 194 198
updated :	
22 32 33 39 45 48 51 51 74 80 84 88 92 97 101 102 123

### Total Deleted and Inserted Products

In [8]:
total_products_deleted_list.sort()
print(' '.join(map(str,total_products_deleted_list)))

2 3 9 11 12 13 16 18 20 23 26 30 31 37 38 40 44 45 47 56 56 61 62 63 67 68 69 70 72 77 78 79 81 86 89 94 98 99 100 108 114 116 125 129 134 144 147 150 154 160 173 180 185 186 198


In [9]:
total_products_inserted_list.sort()
print(' '.join(map(str,total_products_inserted_list)))

101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 133 134 136 137 139 140 141 142 143 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 159 160 161 162 163 165 166 167 168 169 170 171 172 173 174 174 175 176 177 178 179 180 182 183 184 185 186 187 188 189 190 191 192 193 194 196 197 198 199 200
